In [1]:
import onnxruntime as ort
session = ort.InferenceSession("yolov10m.onnx", providers=["CPUExecutionProvider"])

这里的providers参数根据自己的实际情况设置，我使用的是GPU所以设置的是"CUDAExecutionProvider"。
如果用CPU进行推理，则需设置为"CPUExecutionProvider"；
如果有TensorRT的环境，还可以设置为"TensorrtExecutionProvider"

In [2]:
# 查看模型的输入层和输出层的属性
for input in session.get_inputs():
   print("input name: ", input.name)
   print("input shape: ", input.shape)
   print("input type: ", input.type)

for output in session.get_outputs():
   print("output name: ", output.name)
   print("output shape: ", output.shape)
   print("output type: ", output.type)

input name:  images
input shape:  [1, 3, 640, 640]
input type:  tensor(float)
output name:  output0
output shape:  [1, 300, 6]
output type:  tensor(float)


从上面的打印信息可以知道，模型有一个尺寸为[1, 3, 640, 640]的输入层和一个尺寸分别为[1, 300, 6]的输出层。

In [10]:
import cv2
import numpy as np

In [11]:
# 实现下面数据预处理的函数，preprocess
def preprocess(bgr_image, src_w, src_h, dst_w, dst_h):
   image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
   
   ratio = min(dst_w/src_w, dst_h/src_h)
   border_w = int(round(src_w * ratio / 2) * 2)
   border_h = int(round(src_h * ratio / 2) * 2)
   x_offset = (dst_w - border_w ) // 2  
   y_offset = (dst_h - border_h ) // 2

   image = cv2.resize(image, (border_w, border_h))
   image = cv2.copyMakeBorder(
       image, y_offset, y_offset, x_offset, x_offset, 
       cv2.BORDER_CONSTANT, value=(114, 114, 114)
       )

   image = image.astype(np.float32) / 255.0
   image = np.transpose(image, (2, 0, 1))
   input_tensor = np.expand_dims(image, axis=0)
   return input_tensor, ratio, x_offset, y_offset


整个数据预处理的流程如下：
上面的preprocess函数：
把OpenCV读取的BGR格式图片转换为RGB格式；
计算缩放比例和需要填充的区域，把原始图片进行等比例缩放，对不足的区域进行填充让输入图片的尺寸匹配模型的输入尺寸；
对像素值除以255做归一化操作；
把图像数据的通道顺序由HWC调整为CHW；
扩展数据维度，将数据的维度调整为NCHW。

In [12]:
#       数据预处理 用opencv读取图片后，首先要做预处理                                                                                                                                                                                                                                                                                                                                                                                                                                                                       数据预处理
image = cv2.imread("DJI_20240616080249_0066_V.jpeg")
print("image shape: ", image.shape)
image_height, image_width, _ = image.shape
_, _, model_height, model_width = session.get_inputs()[0].shape

input_tensor, ratio, x_offset, y_offset = preprocess(image, image_width, image_height, model_width, model_height)


image shape:  (2160, 3840, 3)
